In [1]:
data = "/mnt/d/Data/Instacart/"

1. Build training set, evaluation set and test set

In [2]:
import pandas as pd
import numpy as np

orders = pd.read_csv(data + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32}, engine='c')

In [3]:
orders.drop(['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], inplace=True, axis=1)

In [4]:
# get products already purchased, a.k.a. <user_id, product_id> list in prior list.

order_product_p = pd.read_csv(data + 'order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8}, engine='c', usecols=['order_id', 'product_id'])

In [5]:
user_order_product_p = pd.merge(orders, order_product_p, on='order_id').drop('eval_set', axis=1)

In [6]:
user_product_p = user_order_product_p.drop('order_id', axis=1).drop_duplicates()
user_product_p.head()

,user_id,product_id
0,1,196
1,1,14084
2,1,12427
3,1,26088
4,1,26405


In [7]:
train = pd.merge(orders.loc[orders['eval_set']=='train'], user_product_p).drop('eval_set', axis=1)
train.head()

,order_id,user_id,product_id
0,1187899,1,196
1,1187899,1,14084
2,1187899,1,12427
3,1187899,1,26088
4,1187899,1,26405


In [8]:
traingroundtruth = pd.read_csv(data + 'order_products__train.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8}, engine='c', usecols=['order_id', 'product_id', 'reordered'])
train = train.merge(traingroundtruth, how='left', on=['order_id', 'product_id'])

In [9]:
label = train['reordered'].copy()
label[label!=1]=0
train.drop('reordered', axis=1, inplace=True)
train['label']=label.astype(np.int8)

In [18]:
user = train.user_id.drop_duplicates()
seed = pd.DataFrame({
    'user_id': user,
    'seed': np.random.randint(0, 5, user.shape)
})

In [21]:
train = train.merge(seed, on='user_id')
train.head()

,order_id,user_id,product_id,label,seed
0,1187899,1,196,1,1
1,1187899,1,14084,0,1
2,1187899,1,12427,0,1
3,1187899,1,26088,1,1
4,1187899,1,26405,1,1


In [23]:
train.to_csv("/mnt/d/Data/Instacart/train.tsv", sep='\t', index=False)

In [20]:
test = pd.merge(orders.loc[orders['eval_set']=='test'], user_product_p).drop('eval_set', axis=1)
test.to_csv("/mnt/d/Data/Instacart/test.tsv", sep='\t', index=False)